In [175]:
import circuitgraph as cg
import pandas as pd
import networkx as nx
import os
import math
from circuitgraph.io import genus_flops, generic_flop, dc_flops
from circuitgraph import BlackBox

### Making Stdcell library dict

In [288]:
stdcell45nmlib = pd.read_excel("../Data/stdcell45nm.xlsx")

In [289]:
stdcell45nmlib

,macro,area,l_power,p1_cap,p2_cap
0,and,1.064,25.07,0.9181,0.9746
1,or,1.064,22.69,0.9468,0.9419
2,nand,0.798,17.39,1.5990,1.6642
3,nor,0.798,21.20,1.7145,1.6513
4,not,0.532,14.35,1.7002,0.0000
5,xor,1.596,36.16,2.2321,2.4115
6,xnor,1.596,36.44,2.2328,2.5736
7,fflopd,4.522,79.11,0.9497,1.1403
8,buf,0.798,21.44,0.9747,0.0000


In [290]:
macro_area_dict = dict(zip(stdcell45nmlib.macro, stdcell45nmlib.area))

In [363]:
circuits_path = "../Data/Circuits/"
circuits_list  = os.listdir(circuits_path)

In [346]:
def gate_count(circuit_name):
    
    '''Pass the full path '''
    
    gates_count_dict = dict()
    circuit = cg.from_file(circuit_name, blackboxes=genus_flops + dc_flops + [generic_flop])
    list_of_macros = list(stdcell45nmlib.macro)
    for macro in list_of_macros:
        if macro=='fflopd':
            macro = 'bb_output'              
            
        cnt = len(circuit.filter_type(macro))
        
        if macro == 'buf':
            if circuit_name != '../Data/Circuits/s27_unrolled_4.v':
                cnt = 0
        if macro=='bb_output':
            macro='fflopd'
            
        gates_count_dict.update({macro:cnt})
    return gates_count_dict


In [347]:
def total_gates_in_circuit(circuit_name):
    """ Pass the full path..."""
    gates_count_dict = gate_count(circuit_name)
    total = 0
    for i,j in gates_count_dict.items():
        total = total + j
    return total

In [348]:
def get_grid_size(circuit_name):
    '''
    Get a tuple containing grid size for our graph in chip design environment.
    
    '''
    get_circuit_gate_dict = gate_count(circuit_name)
    
    total_area_grid = 0
    for macro,count in get_circuit_gate_dict.items():
        total_area_grid = total_area_grid + (count * math.ceil(macro_area_dict[macro]))

    float_root_area_row = math.ceil(math.sqrt(total_area_grid)) +1
    return (float_root_area_row, float_root_area_row)

In [371]:
def addCircuitInfo(netlist):
    """
    This will take circuit as a parameter.
    and will fill our circuit_info table about gates and their counts and other things.
    """
    row = list()
    
    actual_path = circuits_path + netlist
    
    row.append(netlist)
    
    gates_count_dict = gate_count(actual_path)
    
    grid_width,grid_height = get_grid_size(actual_path)
    row.append(grid_width)
    row.append(grid_height)
    
    total_gates = total_gates_in_circuit(actual_path)
    row.append(total_gates)
    
    for macro,cnt in gates_count_dict.items():
        row.append(cnt)
        
    circuit_area_df.loc[circuit_area_df.shape[0]] = row

In [441]:
# circuit_area_df.to_excel('circuit_information.xlsx')

In [442]:
# circuit = cg.from_file('../Data/Circuits/s13207.v', blackboxes = [generic_flop] + genus_flops + dc_flops)

In [446]:
# df = pd.read_excel('../Data/circuit_information.xlsx')

In [447]:
circuit_information = pd.read_excel('../Data/circuit_information.xlsx')

In [752]:
circuit_information

,circuit_name,grid_width,grid_height,total_gates,n_and,n_or,n_nand,n_nor,n_not,n_xor,n_xnor,n_fflopd,n_buf
0,adder.v,58,58,2162,1017,3,0,0,1142,0,0,0,0
1,arbiter.v,189,189,23189,11776,5,16,0,11392,0,0,0,0
2,b17_Cg.v,162,162,25648,0,0,17938,2538,5172,0,0,0,0
3,b18_Cg.v,250,250,61761,0,0,43413,5277,13071,0,0,0,0
4,b19_Cg.v,346,346,118416,0,0,83206,10631,24579,0,0,0,0
5,b20_Cg.v,101,101,9891,0,0,7204,678,2009,0,0,0,0
6,b22_Cg.v,124,124,14894,0,0,10803,973,3118,0,0,0,0
7,banyan_128.v,116,116,13056,0,0,9216,0,3840,0,0,0,0
8,banyan_16.v,30,30,816,0,0,576,0,240,0,0,0,0
9,banyan_256.v,176,176,30464,0,0,21504,0,8960,0,0,0,0


In [753]:
def dict_of_node_to_attributes(circuit_name):
    '''
    Given circuit now make a dictionary for like node and it's attributes
    eg. N1 : {nand, power, area, capacitance_1, capacitance_2}
    '''
    macros_list = list(stdcell45nmlib.macro)
    macros_list.append('bb_input')
    
    circuit = cg.from_file(f'../Data/Circuits/{circuit_name}', blackboxes=[generic_flop] + genus_flops + dc_flops)
    mapping = dict()
    {'node_name':{'type' : 'and', 'area' : '1.20', 'power':'98.0', 'cap_A':'1.22', 'cap_B':'2.011'}}
    for node in circuit.nodes():
        info = dict()
        
        if circuit.type(node) in macros_list:
            info.update({'type': circuit.type(node)})
            info.update({'area': })
            info.update({'power':})
            info.update({'p1_cap':})
            info.update({'p2_cap':})


            mapping.update({node : info})

    return mapping







    

    
    
    

SyntaxError: expression expected after dictionary key and ':' (926922943.py, line 17)

In [754]:
dict_of_node_to_attributes('c17.v')

{'N23': {'type': 'nand'},
 'N22': {'type': 'nand'},
 'N16': {'type': 'nand'},
 'N10': {'type': 'nand'},
 'N11': {'type': 'nand'},
 'N19': {'type': 'nand'}}

In [755]:
chip = ChipDesignGraph('s13207.v')

In [756]:
chip = chip.get_circuit()

In [795]:
len(chip.filter_type(['and','or','nand','nor','not','xnor','xor' ,'bb_output']))

1086

In [537]:
class ChipDesignGraph:
    def __init__(self,circuit_name):
        self.circuit = cg.from_file(f'../Data/Circuits/{circuit_name}', blackboxes= genus_flops + dc_flops + [generic_flop])
        self.circuit_name = self.circuit.name+'.v'
        self.grid_size = get_grid_size(f'../Data/Circuits/{circuit_name}')
        self.design_graph = nx.grid_2d_graph(*self.grid_size)
        self.node_attributes = {}
        self.edge_attributes = {}
    

    def get_circuit(self):
        return self.circuit  

    
        

    def add_node_attributes(self,  node_name_in):
        """
        Add attributes to each node in the graph.
        node_data: Dictionary mapping node IDs to attribute dictionaries.
        """

        node_data = {node_name_in: self.circuit.type(node_name_in)}
        
        self.node_attributes.update(node_data)
        for node, data in node_data.items():
            self.design_graph.nodes[node].update(data)

    
    def add_edge_attributes(self, edge_data):
        """
        Add attributes to edges in the graph.
        edge_data: Dictionary mapping edge tuples (u, v) to attribute dictionaries.
        """
        self.edge_attributes.update(edge_data)
        for edge, data in edge_data.items():
            self.design_graph.edges[edge].update(data)

    
    def DrawGraph(self):
        return nx.draw_spectral(self.design_graph)

In [452]:
def add_to_dictionary(circuit_name):
    aman = ChipDesignGraph(f'../Data/Circuits/{circuit_name}')
    c = aman.get_circuit()
    
    ls_and = set()
    ls_or = set()
    ls_nand = set()
    ls_nor = set()
    ls_not = set()
    ls_buf = set()
    ls_fflopd = set()
    
    
    for node in c.nodes():
        if c.type(node) == 'and':
            ls_and.add(node)
        if c.type(node) == 'or':
            ls_or.add(node)
        if c.type(node) == 'nand':
            ls_nand.add(node)
        if c.type(node) == 'nor':
            ls_nor.add(node)
        if c.type(node) == 'not':
            ls_not.add(node)
        if c.type(node) == 'buf':
            ls_buf.add(node)
        if c.type(node) == 'bb_input':
            if '_reg.D' in node:
                ls_fflopd.add(node)
            
        
    cr = {circuit_name : {
        'nodes' : c.nodes(),
        'n_and' : ls_and,
        'n_or' : ls_or,
        'n_nand' : ls_nand,
        'n_nor' : ls_nor,
        'n_not' : ls_not,
        'n_buf' : ls_buf,
        'n_fflopd' : ls_fflopd,
    }}
    
    circuit_dict.update(cr)
    circuit_dict